# Library Imports

In [24]:
# To install OpenCV: type "conda install opencv" on terminal
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import glob
import random

from skimage.feature import corner_harris, corner_peaks
from sklearn import linear_model
from skimage.transform import warp, AffineTransform
from skimage.measure import ransac
from skimage.util.shape import view_as_windows
from numpy.linalg import norm as normalize
import cv2

if not os.path.exists("Q4"):
    os.makedirs("Q4")



# Task 4.3 (a) - Finding Matching Points (SIFT)

In [16]:
if __name__ == '__main__':
    chapel1 = cv2.imread('Files/chapel1.png',0)
    chapel2 = cv2.imread('Files/chapel2.png',0) 

    sift = cv2.SIFT()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(chapel1,None)
    kp2, des2 = sift.detectAndCompute(chapel2,None)

    # FLANN parameters
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)

    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)

    good = []
    pts1 = []
    pts2 = []

    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.8*n.distance:
            good.append(m)
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)
    
    chapel1 = cv2.cvtColor(chapel1,cv2.COLOR_GRAY2BGR)
    chapel2 = cv2.cvtColor(chapel2,cv2.COLOR_GRAY2BGR)
    
    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)
    
    for pt1, pt2 in zip(pts1,pts2): 
        x1,y1 = pt1
        cv2.circle(chapel1,(x1,y1),2,(0,0,255),-1)
        x2,y2 = pt2
        cv2.circle(chapel2,(x2,y2),2,(0,0,255),-1)

    # Plotting matched points on figure
    plt.imshow(chapel1)
    fn = 'chapel1_SIFT.png'
    plt.savefig('Q4/' + fn, dpi=200)
    plt.close()
    
    plt.imshow(chapel2)
    fn = 'chapel2_SIFT.png'
    plt.savefig('Q4/' + fn, dpi=200)
    plt.close()

# Task 4.3 (b) - Estimating Fundamental Matrix

** RANSAC **

(i) Finding Fundamental Matrix, Inliers and Outliers

In [17]:
pts1 = np.float32(pts1)
pts2 = np.float32(pts2)    
F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_RANSAC)
   
#Keep record of outlier points
outliers1 = pts1[mask.ravel()==0]
outliers2 = pts2[mask.ravel()==0]

#Select only inlier points
pts1 = pts1[mask.ravel()==1]
pts2 = pts2[mask.ravel()==1]

(ii) Normalizing Fundamental Matrix

In [18]:
print "Fundamental Matrix :" 
print F
print "\n"

F = np.array(F)
norm = normalize(F)
F_norm = np.zeros(F.shape)

for i in range(len(F)):
    for j in range(len(F[i])):
        F_norm[i][j] = F[i][j]/norm
        
print "Normalized Fundamental Matrix :"
print F_norm

Fundamental Matrix :
[[ -1.41310716e-07   1.23557572e-04  -1.69133474e-02]
 [ -1.18458902e-04  -3.44997180e-06  -4.69073921e-02]
 [  1.73720033e-02   4.35977543e-02   1.00000000e+00]]


Normalized Fundamental Matrix :
[[ -1.40980578e-07   1.23268909e-04  -1.68738334e-02]
 [ -1.18182151e-04  -3.44191177e-06  -4.67978043e-02]
 [  1.73314178e-02   4.34958986e-02   9.97663740e-01]]


(iii) Plotting Outliers

In [29]:
chapel1 = cv2.imread('Files/chapel1.png')
chapel2 = cv2.imread('Files/chapel2.png')

outliers1 = np.int32(outliers1)
outliers2 = np.int32(outliers2)

for i in  range(len(outliers1)):
            
    x1,y1 = outliers1[i]
    cv2.circle(chapel1,(x1,y1),3,(0,255,0),-1)
    
    x2,y2 = outliers2[i]
    cv2.circle(chapel2,(x1,y2),3,(0,255,0),-1)
        
    plt.imshow(chapel1)
    fn = 'chapel1_outliers.png'
    plt.savefig('Q4/' + fn, dpi=200)
    plt.close()
    
    plt.imshow(chapel2)
    fn = 'chapel2_outliers.png'
    plt.savefig('Q4/' + fn, dpi=200)
    plt.close()

# Task 4.3 (c) - Drawing Epipolar Lines

In [27]:
# this function draws the first 7 epilines on img1 based on the points in img2; lines = corresponding epilines
def drawlines(img1,img2,lines,pts1,pts2):

    r,c,_ = img1.shape
    
    for i in range(7):
        n = random.randint(0,len(pts1))
        #color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -lines[n][2]/lines[n][1] ])
        x1,y1 = map(int, [c, -(lines[n][2]+lines[n][0]*c)/lines[n][1] ])
        cv2.line(img1, (x0,y0), (x1,y1), (255,0,0),1)
        cv2.circle(img1,tuple(np.int32(pts1[n])),3,(0,255,0),-1)
        cv2.circle(img2,tuple(np.int32(pts2[n])),3,(0,255,0),-1)
    return img1,img2

In [30]:
chapel1 = cv2.imread('Files/chapel1.png')
chapel2 = cv2.imread('Files/chapel2.png')

# Find epilines corresponding to points in right image (second image) and
# drawing its lines on left image

lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2,F_norm)
lines1 = lines1.reshape(-1,3)
img5,img6 = drawlines(chapel1,chapel2,lines1,pts1,pts2)

# Find epilines corresponding to points in left image (first image) and
# drawing its lines on right image
lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F_norm)
lines2 = lines2.reshape(-1,3)
img3,img4 = drawlines(chapel2,chapel1,lines2,pts2,pts1)

fig = plt.figure()
ax = fig.add_subplot(121), plt.imshow(img5)
plt.axis('off')
ax = fig.add_subplot(122), plt.imshow(img3)
plt.axis('off')
fig.savefig('Q4/epiline.png', dpi = 200)


# Appendix (unused)

Automatic matching

In [ ]:
def HarrisCornerCV(frame, numCorners, quality, minDist):
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    #dst = cv2.cornerHarris(gray,2,3,0.04)
    #img[dst>0.01*dst.max()]=[0,0,255]
    #cv2.imshow('dst',img)
    corners = cv2.goodFeaturesToTrack(gray, numCorners, quality, minDist)
    corners = np.int0(corners)
    
    return corners

# this function creates an image patch with size LxL centered at coord. 
# NOTE: L must be odd, so that centre of window can be at coord
def CreatePatch(coord, img, L):
    window = view_as_windows(img, (L,L))
    patch = window[coord[1] - L/2, coord[0] - L/2]    
    
    return patch

# this function creates a search window of size LxL centered at refCoord, and returns all feature points in 
# non-stabilized image that are within that window
# NOTE: L must be odd, so that centre of window can be at coord
def SearchFP(refCoord,imgFP,L):
    searchedFP = []
    for imgCoord in imgFP:
        if abs(imgCoord[0]-refCoord[0]) <= L/2 and abs(imgCoord[1]-refCoord[1]) <= L/2:
            searchedFP.append(imgCoord)
            
    return searchedFP

# this function creates template matching of 2 images and output the min value of SSD
def TemplateMatchSSD(img, template):
    method = eval('cv2.TM_SQDIFF')

    # Apply template Matching
    res = cv2.matchTemplate(img,template,method)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    
    return min_val
            
    from sklearn import linear_model


In [228]:
if __name__ == '__main__':
    path = 'Files/'
    imagelist = []

    # reading images and storing filename
    for filename in glob.glob(os.path.join(path, '*.png')): 
        im = cv2.imread(filename)
        fn = filename.split('/')[-1].split('.')[0]
        imagelist.append((fn, im))

    refFP, imgFP = [], []
    refFilename, refFrame = imagelist[0]
    refGray = cv2.cvtColor(refFrame,cv2.COLOR_BGR2GRAY)
    
    # Creating Feature Points for Reference Image (chapel 1)
    corners = HarrisCornerCV(refFrame, 100, 0.01, 10)
    for i in corners:
        x,y = i.ravel()
        refFP.append([x,y])
    
    # Creating array of Matched Feature Points for and drawing all Matched Feature Points in Q4 folder 
    filename, frame = imagelist[1]
    imgGray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
    # Creating Feature Points for Non-Stabilised Image
    corners = HarrisCornerCV(frame, 100, 0.01, 10)
    for i in corners:
        x,y = i.ravel()
        imgFP.append([x,y])
    
    matchedCoord = [] # array to store matched feature points
        
    for refCoord in refFP:
        # Finding feature points in non-stabilised image that are within search window
        searchedFP = SearchFP(refCoord, imgFP, 15) #window bigger than min_dist between FP
        if len(searchedFP) == 0:
             continue # if no feature points were found in search window, don't register refCoord to matchedCoord
        else:    
            template = CreatePatch(refCoord, refGray, 10)
            SSD = [] # array to store SSD of searchedPatch vs stabilised image patch
            for searchedCoord in searchedFP:
                img = CreatePatch(searchedCoord, imgGray,10)
                min_val = TemplateMatchSSD(img, template)
                SSD.append(min_val)
                
            # matching ref FP to with FP in non-stabilised image
            SSD = np.array(SSD)
            idx = np.argmin(SSD)
            matchedCoord.append([refCoord, searchedFP[idx]])
        
    print str(len(matchedCoord)) + " Matched FPs found"
        
    for i in range(len(matchedCoord)):
            
        xR,yR = matchedCoord[i][0]
        cv2.circle(refFrame,(xR,yR),3,(0,0,255),-1)
        xI,yI = matchedCoord[i][1]
        cv2.circle(frame,(xI,yI),3,(0,0,255),-1)
     
    plt.imshow(frame)
    fn = 'chapel2_autoMatch.png'
    plt.savefig('Q4/' + fn, dpi=200)
    plt.close()
        
    plt.imshow(refFrame)
    fn = 'chapel1_autoMatch.png'
    plt.savefig('Q4/' + fn, dpi=200)
    plt.close()
    
    

38 Matched FPs found


In [ ]:
pts1, pts2 = [], []

put chapel1.jpg feature points into pts1, and chapel2.jpg feature points into pts2
for row in matchedCoord:
    pts1.append(row[0])
    pts2.append(row[1])
    
pts1 = np.float32(pts1)
pts2 = np.float32(pts2)    
F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_RANSAC)
   
#Keep record of outlier points
outliers1 = pts1[mask.ravel()==0]
outliers2 = pts2[mask.ravel()==0]

#Select only inlier points
pts1 = pts1[mask.ravel()==1]
pts2 = pts2[mask.ravel()==1]
